### Metaprogramming - Application 1

Are you tired of writing boiler-plate code like this:

In [1]:
class Point2D:
    __slots__ = ('_x', '_y')
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y) == (other.x, other.y)
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'
    
    def __str__(self):
        return f'({self.x}, {self.y})'
        
class Point3D:
    __slots__ = ('_x', '_y', '_z')
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    @property
    def z(self):
        return self._z
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y, self.z) == (other.x, other.y, other.z)
    
    def __hash__(self):
        return hash((self.x, self.y, self.z))

    def __repr__(self):
        return f'Point2D({self.x}, {self.y}, {self.z})'
    
    def __str__(self):
        return f'({self.x}, {self.y}, {self.z})'


It's basically the opposite of DRY!

Let's try to solve this problem using metaclasses (because we might care about inheritance).

First we are going to define our fields using a class attribute, like so:

In [2]:
class Point2D:
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
    
class Point3D:
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

For now, we'll keep the `__init__` in our classes themselves, but we'll come back to that later.

Next we are going to define a metaclass that will create the properties and slots, as well as implement the `__eq__`, `__hash__`, `__repr__` and `__str__` methods.

In [3]:
class SlottedStruct(type):
    def __new__(cls, name, bases, class_dict):
        cls_object = super().__new__(cls, name, bases, class_dict)
        
        # setup the __slots__
        setattr(cls_object, '__slots__', [f'_{field}' for field in cls_object._fields])
            
        # create read-only property for each field
        for field in cls_object._fields:
            slot = f'_{field}'
            # this will not work!
            # remember about how closures work! The free variable is resolved when the function is called!
            #     setattr(cls_object, field, property(fget=lambda self: getattr(self, slot)))
            # so instead we have to use this workaround, by specifying the slot as a defaulted argument
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, attrib)))

        return cls_object

Let's see how this is looking so far:

In [4]:
class Person(metaclass=SlottedStruct):
    _fields = ['name', 'age']
    
    def __init__(self, name, age):
        self._name = name
        self._age = age

In [5]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '_fields': ['name', 'age'],
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name', '_age'],
              'name': <property at 0x7fc7d0255f48>,
              'age': <property at 0x7fc7d0255f98>})

As you can see we have `__slots__` defined, and properties for `name` and `age`. Let's try it out:

In [6]:
p = Person('Alex', 19)

In [7]:
p.name

'Alex'

In [8]:
p.age

19

So far so good, now let's continue implementing the rest of the functions:

In [9]:
class SlottedStruct(type):
    def __new__(cls, name, bases, class_dict):
        cls_object = super().__new__(cls, name, bases, class_dict)
        
        # setup the __slots__
        setattr(cls_object, '__slots__', [f'_{field}' for field in cls_object._fields])
            
        # create read-only property for each field
        for field in cls_object._fields:
            slot = f'_{field}'
            # this will not work!
            #     setattr(cls_object, field, property(fget=lambda self: getattr(self, slot)))
            # Remember about how closures work! The free variable is resolved when the function is called!
            # So instead we have to use this workaround, by specifying the slot as a defaulted argument
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, attrib)))

        # create __eq__ method
        def eq(self, other):
            if isinstance(other, cls_object):
                # ensure each corresponding field is equal
                self_fields = [getattr(self, field) for field in cls_object._fields]
                other_fields = [getattr(other, field) for field in cls_object._fields]
                return self_fields == other_fields
            return False
        setattr(cls_object, '__eq__', eq)

        # create __hash__ method
        def hash_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            return hash(tuple(field_values))
        setattr(cls_object, '__hash__', hash_)
        
        # create __str__ method
        def str_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            field_values_joined = ', '.join(map(str, field_values))  # make every value a string
            return f'{cls_object.__name__}({field_values_joined})'
        setattr(cls_object, '__str__', str_)
        
        # create __repr__ method
        def repr_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            field_key_values = (f'{key}={value}' for key, value in zip(cls_object._fields, field_values))
            field_key_values_str = ', '.join(field_key_values)
            return f'{cls_object.__name__}({field_key_values_str})'
        setattr(cls_object, '__repr__', repr_)
        
        return cls_object

In [10]:
class Person(metaclass=SlottedStruct):
    _fields = ['name']
    
    def __init__(self, name):
        self._name = name

Let's try this out:

In [11]:
type(Person)

__main__.SlottedStruct

In [12]:
p1 = Person('Alex')
p2 = Person('Alex')

In [13]:
type(p1), isinstance(p1, Person)

(__main__.Person, True)

In [14]:
p1 == p2

True

In [15]:
hash(p1), hash(p2)

(-4434760416215847140, -4434760416215847140)

In [16]:
repr(p1)

'Person(name=Alex)'

In [17]:
str(p1)

'Person(Alex)'

And now, we can use this metaclass for any of our other classes too that need to follow the same pattern: slots for all the fields, read-only properties for all the fields, and equality, hashing, repr and str as implemented.

In [18]:
class Point2D(metaclass=SlottedStruct):
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
class Point3D(metaclass=SlottedStruct):
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [19]:
p1 = Point2D(1, 2)
p2 = Point2D(1, 2)
p3 = Point2D(0, 0)

In [20]:
repr(p1), str(p1), hash(p1), p1.x, p1.y

('Point2D(x=1, y=2)', 'Point2D(1, 2)', 3713081631934410656, 1, 2)

In [21]:
repr(p2), str(p2), hash(p2), p2.x, p2.y

('Point2D(x=1, y=2)', 'Point2D(1, 2)', 3713081631934410656, 1, 2)

In [22]:
p1 is p2, p1 == p2

(False, True)

In [23]:
p1 is p3, p1 == p3

(False, False)

And `Point3D` works exactly the same:

In [24]:
p1 = Point3D(1, 2, 3)
p2 = Point3D(1, 2, 3)
p3 = Point3D(0, 0, 0)

In [25]:
p1.x, p1.y, p1.z

(1, 2, 3)

In [26]:
p1 == p2, p1 == p3

(True, False)

Here's an additional twist!

I don't like writing `metaclass=SlottedStruct` every time - so I'm going to use a class decorator to do that for me!!

We already know that a class has properties named `__name__` and `__dict__`.

An additional property it has is `__bases__`:

In [27]:
Point2D.__name__, Point2D.__bases__, Point2D.__dict__

('Point2D',
 (object,),
 mappingproxy({'__module__': '__main__',
               '_fields': ['x', 'y'],
               '__init__': <function __main__.Point2D.__init__(self, x, y)>,
               '__dict__': <attribute '__dict__' of 'Point2D' objects>,
               '__weakref__': <attribute '__weakref__' of 'Point2D' objects>,
               '__doc__': None,
               '__slots__': ['_x', '_y'],
               'x': <property at 0x7fc7d0256778>,
               'y': <property at 0x7fc7d02567c8>,
               '__eq__': <function __main__.SlottedStruct.__new__.<locals>.eq(self, other)>,
               '__hash__': <function __main__.SlottedStruct.__new__.<locals>.hash_(self)>,
               '__str__': <function __main__.SlottedStruct.__new__.<locals>.str_(self)>,
               '__repr__': <function __main__.SlottedStruct.__new__.<locals>.repr_(self)>}))

So, our class decorator will need to take the class, and rebuild it, but specifying the metaclass we want to use:

In [28]:
def struct(cls):
    return SlottedStruct(cls.__name__, cls.__bases__, dict(cls.__dict__))

In [29]:
@struct
class Point2D:
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y = y

In [30]:
type(Point2D)

__main__.SlottedStruct

In [31]:
p = Point2D(1, 2)

In [32]:
type(p)

__main__.Point2D

In [33]:
p.x, p.y

(1, 2)

In [34]:
repr(p)

'Point2D(x=1, y=2)'

All this takes a little bit of getting used to, but the basic concepts are not particularly difficult. The applications thereof do mean you have to use just about everything you've learned about Python in this series!

This was a good exercise to see metaprogramming in action, but as far as this example is concerned we have a much better alternative, starting in Python 3.7 - **dataclasses**.

We'll come back to those later.